In [ ]:
!pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import year, month
from snowflake.snowpark.functions import col, avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException

In [ ]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "MEIXI_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }

In [ ]:
session= Session.builder.configs(parameters).create()

General CPI Correlation, monthly (full)

no p-values

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
#final_df.to_pandas()

,FECHAEMISION,DIASCOMPENSACION,YEAR,MONTH
0,2018-07-06,74.0,2018,7
1,2018-07-27,-17.0,2018,7
2,2018-07-27,130.0,2018,7
3,2018-07-27,-7.0,2018,7
4,2023-10-26,58.0,2023,10
...,...,...,...,...
5032,2008-07-08,26.0,2008,7
5033,2006-04-04,38.0,2006,4
5034,2006-08-04,13.0,2006,8
5035,2013-03-13,5.0,2013,3


In [ ]:
df_cpi = session.sql('select * from CPI_COPY')

In [ ]:
#df_cpi.to_pandas()

,ExtractionDate,Reg,Descripción series,Date (varchar),Date (#),IPC
0,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Jan.2009,2009-01-01,59.05
1,2024-03-21 21:55:32.380,2.0,IPC sin volátiles,Jan.2009,2009-01-01,59.22
2,2024-03-21 21:55:32.380,3.0,IPC Servicios sin volátiles,Jan.2009,2009-01-01,56.14
3,2024-03-21 21:55:32.380,4.0,IPC Bienes sin volátiles,Jan.2009,2009-01-01,62.38
4,2024-03-21 21:55:32.380,5.0,IPC volátil,Jan.2009,2009-01-01,58.71
...,...,...,...,...,...,...
1443,2024-03-21 21:55:32.380,4.0,IPC Bienes sin volátiles,Jan.2024,2024-01-01,100.50
1444,2024-03-21 21:55:32.380,5.0,IPC volátil,Jan.2024,2024-01-01,101.87
1445,2024-03-21 21:55:32.380,6.0,IPC Alimentos volátiles,Jan.2024,2024-01-01,103.55
1446,2024-03-21 21:55:32.380,7.0,IPC Energía volátiles,Jan.2024,2024-01-01,102.04


In [ ]:
df_cpi = df_cpi.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_cpi.columns])

In [ ]:
general_cpi_df = df_cpi[df_cpi['Descripción series'] == 'IPC general (empalme BCCh)\xa0']
#general_cpi_df.to_pandas()

In [ ]:
# Extract year and month for 'FechaEmision' in df_v
final_df = final_df.withColumn('Year', year('FECHAEMISION'))
final_df = final_df.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'Date (#)' in general_cpi_df
general_cpi_df = general_cpi_df.withColumn('Year', year('Date (#)'))
general_cpi_df = general_cpi_df.withColumn('Month', month('Date (#)'))

# Join the dataframes on year and month
df_with_gen_cpi = final_df.join(general_cpi_df, (final_df['Year'] == general_cpi_df['Year']) & (final_df['Month'] == general_cpi_df['Month']))

# Select the desired columns including 'IPC' from general_cpi_df
# Add any other columns you need from df_v and general_cpi_df
df_with_gen_cpi = df_with_gen_cpi.select(final_df['*'], general_cpi_df['IPC'])

# Convert to Pandas DataFrame
# df_with_gen_cpi=df_with_gen_cpi.to_pandas()
# df_with_gen_cpi


In [ ]:
# Upload as table to perform linear regression.

# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_gen_cpi'

# Write the DataFrame to a new table in Snowflake
df_with_gen_cpi.write.mode('overwrite').save_as_table(table_name)

In [ ]:
#tb=session.sql('select * from TABLE_WITH_GEN_CPI ')
#print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_326n_YEAR"', '"l_326n_MONTH"', 'IPC']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "IPC") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "IPC") AS intercept,
    REGR_R2("DIASCOMPENSACION", "IPC") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "IPC") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "IPC") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "IPC") AS syy,
    REGR_SXY("DIASCOMPENSACION", "IPC") AS sxy
FROM
    table_with_gen_cpi;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")


Slope: -3.5823555392684354
Intercept: 371.9852372877329
R-squared: 0.02310572728197876
Sample size: 412515727
SXX: 56828243049.92676
SYY: 31563267692543.816
SXY: -203578971276.79834


General CPI Correlation, monthly (limited)

limited data to 1 obs for each month --still crashes


gets p-values

In [ ]:
# df_v=session.sql('select * from CONSOLIDATED_COPY limit 1000000')
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value)).limit(1)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)


In [ ]:
df_cpi = session.sql('select * from CPI_COPY')
df_cpi = df_cpi.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_cpi.columns])
general_cpi_df = df_cpi[df_cpi['Descripción series'] == 'IPC general (empalme BCCh)\xa0']


In [ ]:
print(general_cpi_df.columns)

['EXTRACTIONDATE', 'REG', '"Descripción series"', '"Date (varchar)"', '"Date (#)"', 'IPC']


In [ ]:
final_df=final_df.to_pandas()
general_cpi_df=general_cpi_df.to_pandas()

In [ ]:
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
general_cpi_df['DATE'] = pd.to_datetime(general_cpi_df['DATE'])

# Create year-month columns
final_df['YEAR_MONTH'] = final_df['FECHAEMISION'].dt.to_period('M')
general_cpi_df['YEAR_MONTH'] = general_cpi_df['"Date (#)"'].dt.to_period('M')

# Merge dataframes on the year-month column
df_with_gen_cpi = pd.merge(final_df, general_cpi_df[['YEAR_MONTH', 'INDEX']], on='YEAR_MONTH', how='left')

# Optionally, if you no longer need the YEAR_MONTH column, you can drop it
df_with_gen_cpi = df_with_gen_cpi.drop(columns=['YEAR_MONTH'])

TypeError: object of type 'Column' has no len()

In [ ]:
# # Ensure 'INDEX' and 'DIASCOMPENSACION' are numeric and handle missing values
# df_with_tci = df_with_tci.dropna(subset=['INDEX', 'DIASCOMPENSACION'])
# df_with_tci['INDEX'] = pd.to_numeric(df_with_tci['INDEX'], errors='coerce')
# df_with_tci['DIASCOMPENSACION'] = pd.to_numeric(df_with_tci['DIASCOMPENSACION'], errors='coerce')
# df_with_tci = df_with_tci.dropna(subset=['INDEX', 'DIASCOMPENSACION'])

# # Prepare the data for the model
# X = df_with_tci['DIASCOMPENSACION']  # Predictor variable
# Y = df_with_tci['INDEX']  # Response variable

# # Add a constant to the model (the intercept)
# X = sm.add_constant(X)

# # Fit the model
# model = sm.OLS(Y, X).fit()

# print(results.summary())

In [ ]:
# # Extract year and month for 'FechaEmision' in df_v
# df_v = df_v.withColumn('Year', year('FECHAEMISION'))
# df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# # Extract year and month for 'Date (#)' in general_cpi_df
# general_cpi_df = general_cpi_df.withColumn('Year', year('Date (#)'))
# general_cpi_df = general_cpi_df.withColumn('Month', month('Date (#)'))

# # Join the dataframes on year and month
# df_with_gen_cpi = df_v.join(general_cpi_df, (df_v['Year'] == general_cpi_df['Year']) & (df_v['Month'] == general_cpi_df['Month']))

# # Select the desired columns including 'IPC' from general_cpi_df
# # Add any other columns you need from df_v and general_cpi_df
# df_with_gen_cpi = df_with_gen_cpi.select(df_v['*'], general_cpi_df['IPC'])

# # Convert to Pandas DataFrame
# df_with_gen_cpi=df_with_gen_cpi.to_pandas()
# df_with_gen_cpi

,FECHAEMISION,DIASCOMPENSACION,l_rmqy_YEAR,l_rmqy_MONTH,IPC
0,2018-07-27,5.0,2018,7,75.93
1,2021-07-21,28.0,2021,7,83.15
2,2021-07-26,-1.0,2021,7,83.15
3,2021-07-26,-14.0,2021,7,83.15
4,2021-07-26,-1.0,2021,7,83.15
...,...,...,...,...,...
460089178,2020-10-15,195.0,2020,10,80.70
460089179,2020-10-15,195.0,2020,10,80.70
460089180,2020-10-15,-5.0,2020,10,80.70
460089181,2020-10-15,-14.0,2020,10,80.70


In [ ]:
#print(df_with_gen_cpi.columns)

Index(['FECHAEMISION', 'DIASCOMPENSACION', 'Year', 'Month', 'IPC'], dtype='object')


In [ ]:
# Group by year and month, then take the first row from each group
# DataFrame now has 1 row per year-month group
df_with_gen_cpi = df_with_gen_cpi.groupby(['Year', 'Month']).first().reset_index()

In [ ]:
pip install statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
# Explanatory model: do not use train test because not predictive

# Convert to numeric and handle non-numeric values
df_with_gen_cpi['IPC'] = pd.to_numeric(df_with_gen_cpi['IPC'], errors='coerce')
df_with_gen_cpi['DIASCOMPENSACION'] = pd.to_numeric(df_with_gen_cpi['DIASCOMPENSACION'], errors='coerce')

# Drop NaN values for a clean analysis
df_with_gen_cpi.dropna(subset=['IPC', 'DIASCOMPENSACION'], inplace=True)

# Prepare the data for modeling
X = df_with_gen_cpi['DIASCOMPENSACION']
Y = df_with_gen_cpi['IPC']
X = sm.add_constant(X)  # adding a constant

# Perform linear regression
model = sm.OLS(Y, X).fit()

# Print the statistics
print(model.summary())

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {model.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {model.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {model.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {model.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(model.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {model.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {model.bse[1]:.3f}")

General CPI Correlation, monthly (limited)

limited data to 10000000 obs

gets p-values

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")
df_cpi = session.sql('select * from CPI_COPY')
df_cpi = df_cpi.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_cpi.columns])
general_cpi_df = df_cpi[df_cpi['Descripción series'] == 'IPC general (empalme BCCh)\xa0']

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
general_cpi_df = general_cpi_df.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
print(general_cpi_df.columns)
general_cpi_df

Index(['EXTRACTIONDATE', 'REG', 'Descripción series', 'Date (varchar)',
       'Date (#)', 'IPC'],
      dtype='object')


,EXTRACTIONDATE,REG,Descripción series,Date (varchar),Date (#),IPC
0,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Jan.2009,2009-01-01,59.05
1,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Feb.2009,2009-02-01,58.28
2,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Mar.2009,2009-03-01,58.30
3,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Apr.2009,2009-04-01,58.21
4,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),May.2009,2009-05-01,58.09
...,...,...,...,...,...,...
176,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Sep.2023,2023-09-01,100.53
177,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Oct.2023,2023-10-01,100.82
178,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Nov.2023,2023-11-01,101.58
179,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Dec.2023,2023-12-01,101.04


In [ ]:
#Convert 'FechaEmision' and 'Date (#)' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
general_cpi_df['Date (#)'] = pd.to_datetime(general_cpi_df['Date (#)'])

# Extract the year from 'FechaEmision' and 'Year'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year
general_cpi_df['YEAR'] = general_cpi_df['Date (#)'].dt.year.astype(int)

# Merge the dataframes on the 'Year' column
df_with_gen_cpi = pd.merge(final_df, general_cpi_df[['YEAR', 'IPC']], on='YEAR', how='inner')

# Drop the extra 'Year' column if you don't need it
df_with_gen_cpi.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_gen_cpi

,FECHAEMISION,DIASCOMPENSACION,IPC
0,2022-08-16,1.0,87.71
1,2022-08-16,1.0,87.96
2,2022-08-16,1.0,89.59
3,2022-08-16,1.0,90.85
4,2022-08-16,1.0,91.93
...,...,...,...
64875,2020-01-08,1257.0,79.65
64876,2020-01-08,1257.0,80.15
64877,2020-01-08,1257.0,80.70
64878,2020-01-08,1257.0,80.59


In [ ]:
# Explanatory model: do not use train test because not predictive

# Convert to numeric and handle non-numeric values
df_with_gen_cpi['IPC'] = pd.to_numeric(df_with_gen_cpi['IPC'], errors='coerce')
df_with_gen_cpi['DIASCOMPENSACION'] = pd.to_numeric(df_with_gen_cpi['DIASCOMPENSACION'], errors='coerce')

# Drop NaN values for a clean analysis
df_with_gen_cpi.dropna(subset=['IPC', 'DIASCOMPENSACION'], inplace=True)

# Prepare the data for modeling
X = df_with_gen_cpi['DIASCOMPENSACION']
Y = df_with_gen_cpi['IPC']
X = sm.add_constant(X)  # adding a constant

# Perform linear regression
model = sm.OLS(Y, X).fit()

# Print the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    IPC   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     250.0
Date:                Mon, 25 Mar 2024   Prob (F-statistic):           3.36e-56
Time:                        09:52:08   Log-Likelihood:            -2.3825e+05
No. Observations:               63655   AIC:                         4.765e+05
Df Residuals:                   63653   BIC:                         4.765e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               70.7911      0.044  

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {model.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {model.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {model.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {model.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(model.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {model.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {model.bse[1]:.3f}")

Model Coefficients:
Intercept: 70.791
Slope (DIASCOMPENSACION): -0.002

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.004
Model Standard Errors:
Standard Error (Intercept): 0.044
Standard Error (DIASCOMPENSACION): 0.000


GDP Deflaote Correlation yearly (full)
no p-values


In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_deflator = session.sql('select * from GDP_DEFLATOR_COPY')
df_deflator = df_deflator.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_deflator.columns])

In [ ]:
print(df_deflator.columns)

['EXTRACTIONDATE', '"Country Name"', 'YEAR', 'DATE', '"deflator(annual%)"']


In [ ]:
# Extract year and month for 'FechaEmision' in df_v
df_v = df_v.withColumn('Year', year('FECHAEMISION'))
df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'DATE' in df_deflator
df_deflator = df_deflator.withColumn('Year', year('DATE'))
df_deflator = df_deflator.withColumn('Month', month('DATE'))

# Join the dataframes on year and month
df_with_deflator = df_v.join(df_deflator, (df_v['Year'] == df_deflator['YEAR']))

# Select the desired columns including 'deflator(annual%)' from df_deflator
# Add any other columns you need from df_v and df_deflator
df_with_deflator = df_with_deflator.select(df_v['*'], df_deflator['deflator(annual%)'])


In [ ]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_deflator'

# Write the DataFrame to a new table in Snowflake
df_with_deflator.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb=session.sql('select * from TABLE_WITH_DEFLATOR ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_oe9b_YEAR"', '"l_oe9b_MONTH"', '"deflator(annual%)"']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "deflator(annual%)") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "deflator(annual%)") AS intercept,
    REGR_R2("DIASCOMPENSACION", "deflator(annual%)") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "deflator(annual%)") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "deflator(annual%)") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "deflator(annual%)") AS syy,
    REGR_SXY("DIASCOMPENSACION", "deflator(annual%)") AS sxy
FROM
    table_with_deflator;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")


Slope: -5.579762383313167
Intercept: 134.14195254811716
R-squared: 0.002872237359005164
Sample size: 395582638
SXX: 3199841666.03109
SYY: 34684830127735.266
SXY: -17854356160.678406


GDP Deflator Correlation, yearly (limited)

limit 10000 observations in each year

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
#final_df.to_pandas()

In [ ]:
df_deflator = session.sql('select * from GDP_DEFLATOR_COPY')

In [ ]:
df_deflator = df_deflator.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_deflator.columns])

In [ ]:
df_deflator.to_pandas()

,EXTRACTIONDATE,Country Name,YEAR,DATE,deflator(annual%)
0,2024-03-23,Chile,1960.0,1960-12-31,NaN
1,2024-03-23,Chile,1961.0,1961-12-31,6.568982
2,2024-03-23,Chile,1962.0,1962-12-31,12.948516
3,2024-03-23,Chile,1963.0,1963-12-31,43.820427
4,2024-03-23,Chile,1964.0,1964-12-31,47.934362
...,...,...,...,...,...
58,2024-03-23,Chile,2018.0,2018-12-31,1.590200
59,2024-03-23,Chile,2019.0,2019-12-31,2.572598
60,2024-03-23,Chile,2020.0,2020-12-31,9.637414
61,2024-03-23,Chile,2021.0,2021-12-31,6.797841


In [ ]:
df_deflator = df_deflator.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
# df_deflator.columns.values

In [ ]:
#Convert 'FechaEmision' and 'DATE' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
df_deflator['DATE'] = pd.to_datetime(df_deflator['DATE'])

# Extract the year from 'FechaEmision' and 'Year'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year
df_deflator['YEAR'] = df_deflator['YEAR'].astype(int)

# Merge the dataframes on the 'Year' column
df_with_deflator = pd.merge(final_df, df_deflator[['YEAR', 'deflator(annual%)']], on='YEAR', how='inner')

# Drop the extra 'Year' column if you don't need it
df_with_deflator.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_deflator

,FECHAEXTRACCION,NUMERODOCUMENTO,NUMEROFOLIO,FECHAEMISION,FECHAVENCIMIENTO,FECHAREGISTRO,BUSINESSPARTNERID,CUENTACONTRATO,CLAVERECONCILIACION,CABECERA_FECHAHORACREACION,...,TRAMOMORA,INHABILITADO,CONCESIONARIA,TIPOCLIENTE,TIPOOPERACION,TIPODOCUMENTO,TIPOPRODUCTO,STATUSCOMPENSACION,STATUSCOMPENSACIONTMP,deflator(annual%)
0,2024-01-15 13:52:32.627,6.360005e+10,1524153.0,2007-01-03,2007-01-22,2007-01-04,0010261705,001100018136,07004FA010AM,2021-04-21 18:16:46,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,5.088757
1,2024-01-15 13:52:32.627,6.360006e+10,1553458.0,2007-01-08,2007-01-26,2007-01-09,0010842264,001100599811,07008FA020AN,2021-04-21 18:17:30,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta Exenta,Tag,Pagada,Pagada,5.088757
2,2024-01-15 13:52:32.627,6.360006e+10,1586519.0,2007-01-11,2007-01-29,2007-01-12,0010716330,001100472003,07010FA030AC,2021-04-21 18:18:14,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,5.088757
3,2024-01-15 13:52:32.627,6.360006e+10,1613180.0,2007-01-17,2007-02-05,2007-01-17,0010286522,001100042992,07016FA040AI,2021-04-21 18:19:09,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,5.088757
4,2024-01-15 13:52:32.627,6.360006e+10,1630220.0,2007-01-17,2007-02-05,2007-01-17,0010204922,001000352446,06017SA041AL,2021-04-21 18:19:09,...,Pago,Cliente Tag,AVN,Natural,Gastos Cobranzas,Boleta Exenta,Tag,Pagada,Pagada,5.088757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169995,2024-01-15 13:52:32.627,6.000068e+11,11480003.0,2018-03-23,2018-04-09,2018-03-22,0011206398,001100832999,180322R4/002,2021-04-25 13:34:06,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Pagada,Pagada,1.590200
169996,2024-01-15 13:52:32.627,5.380065e+10,47725970.0,2018-03-05,2018-03-20,2018-03-05,0011437726,001100993101,18064FA010AJ,2021-04-25 13:13:01,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,1.590200
169997,2024-01-15 13:52:32.627,6.000068e+11,11533558.0,2018-03-29,2018-04-13,2018-03-28,0011485532,001101030828,180328R4/002,2021-04-25 13:40:44,...,Pago,Cliente Tag,TSC,Natural,Intereses,Boleta,Tag,Castigado,Castigado,1.590200
169998,2024-01-15 13:52:32.627,5.380066e+10,48484675.0,2018-05-22,2018-06-06,2018-05-22,0010776798,001100532447,18138FA050BY,2021-04-25 14:54:57,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,1.590200


In [ ]:
df_filtered = df_with_deflator.dropna(subset=['deflator(annual%)', 'DIASCOMPENSACION'])

# Calculate correlation
correlation = df_filtered['deflator(annual%)'].corr(df_filtered['DIASCOMPENSACION'])

# Use statsmodels library for detailed statistics
X = sm.add_constant(df_filtered['deflator(annual%)'])  # Adding a constant term to the predictor
y = df_filtered['DIASCOMPENSACION']

model = sm.OLS(y, X)
results = model.fit()

# Print summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       DIASCOMPENSACION   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     101.0
Date:                Mon, 25 Mar 2024   Prob (F-statistic):           9.48e-24
Time:                        10:17:12   Log-Likelihood:            -1.1702e+06
No. Observations:              163057   AIC:                         2.340e+06
Df Residuals:                  163055   BIC:                         2.340e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               133.2676      1.44

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {results.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {results.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {results.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {results.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(results.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {results.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {results.bse[1]:.3f}")


Model Coefficients:
Intercept: 133.268
Slope (DIASCOMPENSACION): -2.499

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.001
Model Standard Errors:
Standard Error (Intercept): 1.449
Standard Error (DIASCOMPENSACION): 0.249


TPI Correlation Full
no P-value

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_tci = session.sql('select * from TCI_COPY')
df_tci = df_tci.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_tci.columns])

In [ ]:
print(df_tci.columns)

['EXTRACTIONDATE', 'DATE', 'YEAR', 'MONTH', 'GLOSA', 'INDEX', '"Monthly_variation(%)"']


In [ ]:
# Extract year and month for 'FechaEmision' in df_v
df_v = df_v.withColumn('Year', year('FECHAEMISION'))
df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'DATE' in df_tci
df_tci = df_tci.withColumn('Year', year('DATE'))
df_tci = df_tci.withColumn('Month', month('DATE'))

# Join the dataframes on year and month
df_with_tci = df_v.join(df_tci, (df_v['Year'] == df_tci['YEAR'])& (df_v['Month'] == df_tci['Month']))

# Select the desired columns including 'INDEX' from df_tci
# Add any other columns you need from df_v and df_tci
df_with_tci = df_with_tci.select(df_v['*'], df_with_tci['INDEX'])


In [ ]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_tci'

# Write the DataFrame to a new table in Snowflake
df_with_tci.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb=session.sql('select * from TABLE_WITH_TCI ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_ryua_YEAR"', '"l_ryua_MONTH"', 'INDEX']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", INDEX) AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", INDEX) AS intercept,
    REGR_R2("DIASCOMPENSACION", INDEX) AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", INDEX) AS sample_size,
    REGR_SXX("DIASCOMPENSACION", INDEX) AS sxx,
    REGR_SYY("DIASCOMPENSACION", INDEX) AS syy,
    REGR_SXY("DIASCOMPENSACION", INDEX) AS sxy
FROM
    table_with_tci;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")


Slope: -2.404042768024504
Intercept: 331.4957145410913
R-squared: 0.029278469751735172
Sample size: 205847402
SXX: 35982913201.07129
SYY: 7102844808684.804
SXY: -86504462253.48828


Transportation Price Index Correlation (limited)

select 100 bservations from each month in df

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value)).limit(100)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_tci = session.sql('select * from TCI_COPY')

In [ ]:
df_tci = df_tci.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_tci.columns])

In [ ]:
final_df=final_df.to_pandas()
df_tci=df_tci.to_pandas()

In [ ]:
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
df_tci['DATE'] = pd.to_datetime(df_tci['DATE'])

# Create year-month columns
final_df['YEAR_MONTH'] = final_df['FECHAEMISION'].dt.to_period('M')
df_tci['YEAR_MONTH'] = df_tci['DATE'].dt.to_period('M')

# Merge dataframes on the year-month column
df_with_tci = pd.merge(final_df, df_tci[['YEAR_MONTH', 'INDEX']], on='YEAR_MONTH', how='left')

# Optionally, if you no longer need the YEAR_MONTH column, you can drop it
df_with_tci = df_with_tci.drop(columns=['YEAR_MONTH'])

In [ ]:
df_with_tci.head()

,FECHAEXTRACCION,NUMERODOCUMENTO,NUMEROFOLIO,FECHAEMISION,FECHAVENCIMIENTO,FECHAREGISTRO,BUSINESSPARTNERID,CUENTACONTRATO,CLAVERECONCILIACION,CABECERA_FECHAHORACREACION,...,INHABILITADO,CONCESIONARIA,TIPOCLIENTE,TIPOOPERACION,TIPODOCUMENTO,TIPOPRODUCTO,STATUSCOMPENSACION,STATUSCOMPENSACIONTMP,YEAR,INDEX
0,2024-01-15 13:52:32.627,1.840000e+10,3851241.0,2007-08-03,2007-08-21,2007-08-03,0010179009,001000217647,07215FA010BI,2021-04-21 19:06:08,...,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,2007,NaN
1,2024-01-15 13:52:32.627,1.840000e+10,3850751.0,2007-08-03,2007-08-21,2007-08-03,0011086005,001100773476,07215FA010BI,2021-04-21 19:06:08,...,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,2007,NaN
2,2024-01-15 13:52:32.627,1.840001e+10,4753154.0,2007-11-08,2007-11-26,2007-11-08,0010082839,001000103822,07311FA020AC,2021-04-21 19:23:45,...,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,2007,NaN
3,2024-01-15 13:52:32.627,1.840000e+10,3851383.0,2007-08-03,2007-08-20,2007-08-03,0010410514,001100166232,07215FA010BI,2021-04-21 19:06:08,...,Cliente Tag,AVN,Natural,Intereses,Boleta Exenta,Tag,Pagada,Pagada,2007,NaN
4,2024-01-15 13:52:32.627,1.840000e+10,4461167.0,2007-10-08,2007-10-26,2007-10-08,0010772773,001100528423,07280FA020BN,2021-04-21 19:18:32,...,Cliente Tag,AVN,Natural,Gastos Cobranzas,Boleta Exenta,Tag,Pagada,Pagada,2007,NaN


In [ ]:
# Ensure 'INDEX' and 'DIASCOMPENSACION' are numeric and handle missing values
df_with_tci = df_with_tci.dropna(subset=['INDEX', 'DIASCOMPENSACION'])
df_with_tci['INDEX'] = pd.to_numeric(df_with_tci['INDEX'], errors='coerce')
df_with_tci['DIASCOMPENSACION'] = pd.to_numeric(df_with_tci['DIASCOMPENSACION'], errors='coerce')
df_with_tci = df_with_tci.dropna(subset=['INDEX', 'DIASCOMPENSACION'])

# Prepare the data for the model
X = df_with_tci['DIASCOMPENSACION']  # Predictor variable
Y = df_with_tci['INDEX']  # Response variable

# Add a constant to the model (the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(Y, X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       DIASCOMPENSACION   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     101.0
Date:                Mon, 25 Mar 2024   Prob (F-statistic):           9.48e-24
Time:                        10:19:00   Log-Likelihood:            -1.1702e+06
No. Observations:              163057   AIC:                         2.340e+06
Df Residuals:                  163055   BIC:                         2.340e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               133.2676      1.44

<ipython-input-90-4a16b0029e95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_tci['INDEX'] = pd.to_numeric(df_with_tci['INDEX'], errors='coerce')
<ipython-input-90-4a16b0029e95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_tci['DIASCOMPENSACION'] = pd.to_numeric(df_with_tci['DIASCOMPENSACION'], errors='coerce')


In [ ]:
# Print model results
print("Model Coefficients:")
print(f"Intercept: {model.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {model.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {model.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {model.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(model.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {model.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {model.bse[1]:.3f}")

Model Coefficients:
Intercept: 112.081
Slope (DIASCOMPENSACION): -0.005

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.005
Model Standard Errors:
Standard Error (Intercept): 0.186
Standard Error (DIASCOMPENSACION): 0.001


In [ ]:
session.close

<bound method Session.close of <snowflake.snowpark.session.Session object at 0x7c3cb0119cc0>>